In [1]:
import cv2
import numpy as np
import skimage.io as io
from skimage.feature import canny
import matplotlib.pyplot as plt
from skimage.color import rgb2gray , rgb2hsv
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from skimage.measure import find_contours
from skimage.draw import rectangle
from skimage.filters import threshold_otsu, threshold_mean, threshold_li, threshold_isodata, threshold_niblack , median
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin , disk
from skimage.segmentation import flood, flood_fill
%matplotlib inline
%load_ext autoreload
%autoreload 2


In [2]:
def Gamma_Correction(img , c , gamma):
    img2 = np.copy(img)
    img2 = c*np.power(img , gamma)
    return img2

def erosion(img , n,m):
    img2 = np.copy(img)
    for i in range(int(n/2) , img.shape[1]-1-int(n/2) ):
        for j in range(int(m/2) , img.shape[0]-1-int(m/2) ):
            img2[j,i] = np.sum(img[j-int((m)/2) : j+int((m)/2) + n%2 , i-int((n)/2) : i+int((n)/2) + n%2 ])==n*n        
    return img2

def dilation(img , n,m):
    img2 = np.copy(img)
    for i in range(int(n/2) , img.shape[1]-1-int(n/2) ):
        for j in range(int(m/2) , img.shape[0]-1-int(m/2) ):
            img2[j,i] = np.sum(img[j-int((m)/2) : j+int((m)/2) + n%2 , i-int((n)/2) : i+int((n)/2) + n%2 ])>=1        
    return img2

def Median(img , n , m):
    img2 = np.copy(img)
    filt = np.ones((n,m))
    for i in range(int(n/2) , img.shape[1]-int(np.ceil(n/2))):
        for j in range(int(m/2) , img.shape[0]-int(np.ceil(m/2))):
            img2[j,i] = np.median(img[j-int(m/2) : j+int(m/2)+1 , i-int(n/2) : i+int(n/2)+1 ])
    return img2


In [3]:
def multi_color_space_model(rgba_frame , hsv_frame , ycbcr_frame):
    
    # Extracting the blue, red, green and alpha channels
    
    B = rgba_frame[:,:,0]
    G = rgba_frame[:,:,1]
    R = rgba_frame[:,:,2]
    A = rgba_frame[:,:,3]
    
    #Extracting the Hue, Saturation and vue channels
    
    H = hsv_frame[:,:,0]
    S = hsv_frame[:,:,1]
    V = hsv_frame[:,:,2]
    
    # Extracting the Y, Cr and Cb channels
    
    Y = ycbcr_frame[:,:,0]
    Cr = ycbcr_frame[:,:,1]
    Cb = ycbcr_frame[:,:,2]
    
    #     Applying  Thresholding using Log-Chromaticity color space 
    n_r = np.ma.divide(R, G)
    n_b = np.ma.divide(B, G)
    log_rg = np.ma.log( n_r )
    log_bg = np.ma.log( n_b )
    condition_1 = (log_rg>=0.15) & (log_rg<=1.1)
    condition_2 = (log_bg>=-4) & (log_bg<=0.3)
    mask_1 = condition_1 & condition_2
    
    # Extracting masks based on a combination of RGBA, HSV and YCrCb models for skin detection
    
    mask_rgb = (R>95)&(G>40)&(B>20)&(R>G)&(R>B)&(abs(R-G)>15)&(A>15)
    mask_Ycbcr = (Cr > 135)&(Cb>85)&(Y>80)&(Cr <= (1.5862*Cb)+20)&(Cr>=(0.3448*Cb)+76.2069)&(Cr >= (-4.5652*Cb)+234.5652)&(
                  Cr <= (-1.15*Cb)+301.75)&(Cr <= (-2.2857*Cb)+432.85)
    mask_hsv = ((0.0 <= H) & (H <= 50.0))&((0.23 <= S/255) & (S/255 <= 0.68))

    mask1 = mask_rgb & mask_Ycbcr
    mask2 = mask_rgb & mask_hsv
    
    return mask_1
#     return mask_hsv

In [4]:
def moving_area(image1 , image2 , image3):
    diff1 = np.zeros((image1.shape)) 
    diff1 = cv2.absdiff(image1,image2) >= 30
    diff2 = np.zeros((image1.shape))
    diff2 = cv2.absdiff(image1,image3) >= 30
    mask = diff1*diff2
    mask = binary_erosion(mask , disk(5))
    one_positions = np.argwhere(mask >= 1)
    moving_region = np.zeros((image2.shape[0] , image2.shape[1]))
    
    if len(one_positions)> 0:
        Xmin = min(one_positions[:,0])
        Xmax = max(one_positions[:,0])
        Ymin = min(one_positions[:,1])
        Ymax = max(one_positions[:,1]) 
        moving_region[Xmin:Xmax , Ymin:Ymax] = 1
    mask1 = np.zeros((image2.shape[0] , image2.shape[1]))
    mask1[mask] = 1
    cv2.imshow('diff' , mask1)
    cv2.imshow('moving region',moving_region)
    
    return moving_region
    

In [5]:
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

ok_region = False
old_moving_region = -1 

_,frame1 = cap.read()
# converting the image into grayscale image
frame1 = cv2.resize(frame1, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
image3 = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

_,frame2 = cap.read()
# converting the image into grayscale image
frame2 = cv2.resize(frame2, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
image2 = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

fgbg =cv2.createBackgroundSubtractorKNN(history=1000) #,detectShadows=True)

while True:
    
    ret, rgb_frame = cap.read()
    rgb_frame = cv2.resize(rgb_frame, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
    rgb_frame = cv2.GaussianBlur(rgb_frame ,(7,7),cv2.BORDER_DEFAULT) 
    image1 = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2GRAY)

    fgmask = fgbg.apply(rgb_frame)
    dilated_fgmask = binary_dilation(fgmask ,disk(3))
    fgmask = np.zeros(image1.shape)
    fgmask[dilated_fgmask] = 1
    
    cv2.imshow('frame',fgmask)
    
    if(not ok_region):
        ok_region = True
        old_moving_region = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
        
    #face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')    
    #gray = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2GRAY)
    #faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    #for (x,y,w,h) in faces: 
        # To draw a rectangle in a face  
        #cv2.rectangle(rgb_frame,(x,y),(x+w,y+h),(255,255,0),2)   
     
    
    # converting the rgb space to hsv space color
    hsv_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2HSV)
    # converting the rgb space to YCbCr space color
    ycbcr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2YCR_CB)
    
    # converintg the rgb space to rgba spave 
    rgba_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2BGRA)
    
    moving_region = moving_area(image1 , image2 , image3)
    if(np.sum(moving_region) <= 10000):
        moving_region = old_moving_region
    old_moving_region = moving_region
    
    # the final mask to extract the binary image
    mask = multi_color_space_model(rgba_frame , hsv_frame , ycbcr_frame)
    
    binary_frame2 = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    binary_frame2[mask] = 1
  
    #for (x,y,w,h) in faces: 
        # To draw a rectangle in a face  
        #binary_frame2[y-30:y+h+30 , x:x+w] = 0
        
    #removing noise
    eroded_frame = binary_erosion(binary_frame2 , np.ones((3,3)))
    dilated_frame = binary_dilation(eroded_frame , np.ones((10,5)))
    
    binary_frame2 = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    temp = np.copy(binary_frame2).astype("uint8")
    
    binary_frame2[dilated_frame] = 1
    temp[dilated_frame] = 255
    
    contours, hierarchy = cv2.findContours(temp, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    bounding_boxes = list()
    mask_face = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        #if( ( (float(w)/h <= 0.9 and float(h)/w >= 1.3) or (float(h)/w <= 0.5 and float(w)/h >= 1.3 ) )):
        if( ( (float(w)/h <= 0.93) or (float(h)/w <= 0.5 and float(w)/h >= 1.3 ) )):
            mask_face[y:y+h , x:x+w] = 1
            cv2.drawContours(rgb_frame, contour, -1, (0, 255, 0), 3)
    
    #cv2.imshow('binary canny detection', final_frame)
#     cv2.imshow('canny detection', cannyImg )

    frame_with_hand_only = np.zeros((rgb_frame.shape[0] , rgb_frame.shape[1]))
    frame_with_hand_only[(binary_frame2>=1) &  (fgmask >= 1) & (mask_face>=1)] = 1
    
    cv2.imshow('Input', rgb_frame)
    cv2.imshow('Output1', binary_frame2)
    cv2.imshow('Final Output', frame_with_hand_only)
    #cv2.imshow('moving region',moving_region)
    image3 = image2
    image2 = image1
    
    c = cv2.waitKey(1)
    if c == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [6]:
# import numpy as np
# import cv2

# cap = cv2.VideoCapture(0)

# fgbg = cv2.createBackgroundSubtractorMOG2()

# while(1):
#     ret, frame = cap.read()
#     if ret == True:
#         fgmask = fgbg.apply(frame)

#         cv2.imshow('frame',fgmask)
#         k = cv2.waitKey(30) & 0xff
#         if k == 27:
#             break
#     else:
#         break

# cap.release()
# cv2.destroyAllWindows()
